<a href="https://colab.research.google.com/github/AadilRashid/hi-world/blob/master/EndToEnd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/MyDrive/DeepLearning

/content/drive/MyDrive/DeepLearning


## Checking imports

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


## Loading the dataset

In [ ]:
from tensorflow.keras.datasets import cifar10

(x_train,y_train),(x_test,y_test)  = cifar10.load_data()

In [ ]:
size = 10000
x_train_trim = x_train[:size]/255.
y_train_trim = y_train[:size]


x_test_trim = x_test[:int(size*.2)]/255.
y_test_trim = y_test[:int(size*.2)]


x_train_trim.shape,y_train_trim.shape,x_test_trim.shape,y_test_trim.shape

((10000, 32, 32, 3), (10000, 1), (2000, 32, 32, 3), (2000, 1))

In [ ]:
pd.Series(y_train.ravel()).value_counts()

9    5000
8    5000
7    5000
6    5000
5    5000
4    5000
3    5000
2    5000
1    5000
0    5000
dtype: int64

## Defining Model

In [ ]:
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPool2D,Dropout
from tensorflow.keras.regularizers import l1
from tensorflow.keras.models import Sequential

def get_model(decay =0,dropout=.3):
  model = Sequential([
                      
                    Conv2D(32,3,activation ="relu",input_shape = (32,32,3)),
                    MaxPool2D(2),
                    #Dropout(dropout),
                    Conv2D(8,3,activation ="relu"),
                    MaxPool2D(2),
                    #Dropout(dropout),
                    
                    
                    Flatten(),
                    Dense(10,activation="softmax")
  ])

  return model

In [ ]:
def get_test_accuracy(model,x,y):
  print(model.evaluate(x,y))


  

## Calling and Executing the Model

In [ ]:
#initialize a model
model = get_model()
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 8)         2312      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 8)           0         
_________________________________________________________________
flatten (Flatten)            (None, 288)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                2890      
Total params: 6,098
Trainable params: 6,098
Non-trainable params: 0
______________________________________________________

In [ ]:
model.compile(optimizer="adam",loss = "sparse_categorical_crossentropy",metrics =["acc"])


In [ ]:
#define some callbacks
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,CSVLogger
escp =EarlyStopping(patience=15)
mcp = ModelCheckpoint("myModel1",save_weights_only=True)
csvcp=CSVLogger("Traininglogs")

#fitting the model
model.fit(x_train_trim,y_train_trim,validation_split = .15,batch_size=8,verbose=2,callbacks=[escp,mcp])

1063/1063 - 10s - loss: 1.9804 - acc: 0.2781 - val_loss: 1.7942 - val_acc: 0.3460


In [ ]:
get_test_accuracy(model,x_test_trim,y_test_trim)

63/63 [==============================] - 1s 9ms/step - loss: 1.7870 - acc: 0.3475
[1.786990761756897, 0.3474999964237213]


## Using tensorflow Hub model 

In [ ]:
import tensorflow_hub as hub

In [ ]:
m = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/tensorflow/resnet_50/classification/1")
])
m.build([None, 224, 224, 3])  # Batch input shape.


In [ ]:
m.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1001)              25612201  
Total params: 25,612,201
Trainable params: 0
Non-trainable params: 25,612,201
_________________________________________________________________


In [ ]:
from tensorflow.keras.preprocessing import image
img = image.array_to_img(x_test[0])
img = image.smart_resize(np.array(img),(224,224))


pred = m.predict(img[np.newaxis,...])

In [ ]:
np.argmax(pred,axis=1)

array([902])

In [ ]:
with open("/content/drive/MyDrive/DeepLearning/ImageNetLabels.txt","r") as f:
  categories = f.read().splitlines()

In [ ]:
categories[902]

'whiskey jug'